In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# Extraigo la informacion del archivo y guardo cada hoja en un dataframe diferente

df_unido_por_prov_poblacion_hogares = pd.read_parquet('../Archivos_Finales/para_kpi_1_y_3')
df_accesos_totales_sin_outliers = pd.read_parquet('../Archivos_Finales/para_kpi_2')


Decisiones de ajuste:
df_velocidad_sin_Rangos: Dado que tiene muchas columnas de velocidad, tomaremos las 30 más comunes o relevantes en lugar de todas las columnas de velocidad para simplificar el análisis.
df_velocidad_por_prov: Se utilizará para analizar la media de bajada de Mbps por provincia.
df_totales_accesos_por_Tecnologia y df_accesos_por_tecnologia: Se mantendrán ambos, uno para totales y otro para análisis provincial.
df_penetracion_poblacion y df_penetracion_hogares: Son importantes para la penetración de accesos por población y hogares.
df_accesos_por_velocidad: Se utilizará para analizar accesos por rangos de velocidad.
df_ingresos: Se mantendrá para analizar ingresos.
df_mapa_conectividad: Este dataframe parece más adecuado para un análisis geográfico, pero si no es necesario visualizar mapas, podemos omitirlo.
df_poblacion_por_provincia: Se mantendrá para agregar información de la población por provincia y calcular KPIs ajustados por población.

# KPIs de Conectividad a Internet en Argentina

## 1. Incremento de Hogares con Acceso a Internet
**Objetivo:** Aumentar en un 2% el acceso al servicio de internet para el próximo trimestre, cada 100 hogares, por provincia.

**Fórmula:**
$$
KPI = \left( \frac{\text{Nuevo acceso} - \text{Acceso actual}}{\text{Acceso actual}} \right) \times 100
$$

**Donde:**
- "Nuevo acceso" se refiere al número de hogares con acceso a Internet después del próximo trimestre.
- "Acceso actual" se refiere al número de hogares con acceso a Internet en el trimestre actual.

**Ejemplo de uso:**
$$
KPI = \left( \frac{510 - 500}{500} \right) \times 100 = 2\%
$$

Esto indicaría un aumento del 2% en el acceso a Internet en esa provincia para el próximo trimestre.

---

In [10]:
# Filtrar el DataFrame para incluir solo el año 2024 y el trimestre 1
df_2024_trimestre1 = df_unido_por_prov_poblacion_hogares[
    (df_unido_por_prov_poblacion_hogares['Año'] == 2024) & 
    (df_unido_por_prov_poblacion_hogares['Trimestre'] == 1)
]

# Calcular la mediana de "Accesos por cada 100 hogares"
mediana_accesos = df_2024_trimestre1['Accesos por cada 100 hogares'].median()

# Calcular la mediana de "velocidad del internet"
mediana_velocidad = df_2024_trimestre1['Mbps (Media de bajada)'].median()

# Crear una copia del DataFrame filtrado para el nuevo DataFrame de resultados
df_kpi = df_2024_trimestre1.copy()

# Calcular el nuevo valor de acceso con el aumento del 2%
df_kpi['Nuevo_acceso'] = df_kpi['Accesos por cada 100 hogares'] * 1.02

# Calcular el nuevo valor de Mbps con el aumento del 3.5%
df_kpi['Nuevo_Mbps'] = df_kpi['Mbps (Media de bajada)'] * 1.035

# Calcular el KPI para accesos
df_kpi['KPI_Aumento_Acceso'] = ((df_kpi['Nuevo_acceso'] - df_kpi['Accesos por cada 100 hogares']) / df_kpi['Accesos por cada 100 hogares']) * 100
df_kpi['KPI_Aumento_velocidad'] = ((df_kpi['Nuevo_Mbps'] - df_kpi['Mbps (Media de bajada)']) / df_kpi['Mbps (Media de bajada)']) * 100

# Crear un nuevo DataFrame solo con las columnas relevantes
df_kpi_resultado_velocidad = df_kpi[['Provincia', 'Accesos por cada 100 hogares', 'Nuevo_acceso', 'Mbps (Media de bajada)', 'Nuevo_Mbps', 'KPI_Aumento_Acceso', 'KPI_Aumento_velocidad']]

# Mostrar la mediana y el DataFrame de resultados
print(f"Mediana de accesos por cada 100 hogares (2024, Trimestre 1): {mediana_accesos}")
df_kpi_resultado_velocidad.head()

Mediana de accesos por cada 100 hogares (2024, Trimestre 1): 69.065


,Provincia,Accesos por cada 100 hogares,Nuevo_acceso,Mbps (Media de bajada),Nuevo_Mbps,KPI_Aumento_Acceso,KPI_Aumento_velocidad
0,Buenos Aires,81.10,82.7220,159.13,164.69955,2.0,3.5
1,Capital Federal,119.53,121.9206,229.94,237.98790,2.0,3.5
2,Catamarca,68.82,70.1964,91.45,94.65075,2.0,3.5
3,Chaco,46.70,47.6340,107.51,111.27285,2.0,3.5
4,Chubut,84.89,86.5878,20.50,21.21750,2.0,3.5


## 2. Tasa de Adopción de Fibra Óptica
**Objetivo:** Aumentar la tasa de adopción de fibra óptica en un 10% en el próximo trimestre.

**Fórmula:**
$$
KPI = \left( \frac{\text{Número de nuevos usuarios de fibra óptica} - \text{Número de usuarios de fibra óptica (trimestre anterior)}}{\text{Número de usuarios de otras tecnologías (trimestre anterior)}} \right) \times 100
$$

**Ejemplo de uso:**
$$
KPI = \left( \frac{100 - 200}{200} \right) \times 100 = -50\%
$$

Esto indicaría que se han perdido usuarios en el trimestre, lo que podría señalar una necesidad de revisar las estrategias de marketing.

---

In [8]:
# Filtrar el DataFrame para incluir solo el año 2024 y el trimestre 1
df_2024_trimestre1 = df_unido_por_prov_poblacion_hogares[
    (df_unido_por_prov_poblacion_hogares['Año'] == 2024) & 
    (df_unido_por_prov_poblacion_hogares['Trimestre'] == 1)
]

# Calcular la mediana de "Fibra óptica"
mediana_fibra = df_accesos_totales_sin_outliers['Fibra óptica'].median()

# Identificar las 10 provincias con menor acceso a fibra óptica
provincias_menor_fibra = df_accesos_totales_sin_outliers.nsmallest(10, 'Fibra óptica')['Provincia']

# Crear una copia del DataFrame original para el nuevo DataFrame de resultados
df_kpi_fibra = df_accesos_totales_sin_outliers.copy()

# Calcular el nuevo valor de acceso a fibra óptica con el aumento del 10% para provincias con bajo acceso
df_kpi_fibra['Nuevo_Fibra'] = df_kpi_fibra.apply(
    lambda row: row['Fibra óptica'] * 1.10 if row['Provincia'] in provincias_menor_fibra.values else row['Fibra óptica'],
    axis=1
)

# Calcular el KPI de aumento de fibra óptica
df_kpi_fibra['KPI_Aumento_Fibra'] = ((df_kpi_fibra['Nuevo_Fibra'] - df_kpi_fibra['Fibra óptica']) / df_kpi_fibra['Fibra óptica']) * 100

# Crear un nuevo DataFrame solo con las columnas relevantes
df_kpi_resultado_fibra = df_kpi_fibra[['Provincia', 'Fibra óptica', 'Nuevo_Fibra', 'KPI_Aumento_Fibra','ADSL','Cablemodem','Wireless','Total_poblacion']]

# Mostrar los resultados
df_kpi_resultado_fibra.head()

,Provincia,Fibra óptica,Nuevo_Fibra,KPI_Aumento_Fibra,ADSL,Cablemodem,Wireless,Total_poblacion
0,Buenos Aires,24135336.0,24135336.0,0.0,38255033.0,74494554.0,2751144.0,35138106
1,Catamarca,594700.0,594700.0,0.0,808338.0,184834.0,18517.0,429556
2,Chaco,660129.0,660129.0,0.0,1681863.0,1678268.0,147689.0,1142963
3,Chubut,200148.0,220162.8,10.0,1724368.0,1143846.0,730859.0,603120
4,Corrientes,291766.0,320942.6,10.0,1814732.0,1840065.0,156228.0,1197553


## 3. Incremento de la velocidad media por provincia

**Objetivo:** Aumentar en un 3.5% la velocidad media de internet para el próximo trimestre, por provincia.

**Fórmula:**
$$
KPI = \left( \frac{\text{Nueva velocidad} - \text{Velocidad actual}}{\text{Velocidad actual}} \right) \times 100
$$

**Donde:**
- "Nuevo velocidad" se refiere a la velocidad media de Internet después del próximo trimestre.
- "Velocidad actual" se refiere a la velocidad media de Internet en el trimestre actual.

**Ejemplo de uso:**
$$
KPI = \left( \frac{510 - 500}{500} \right) \times 100 = 2\%
$$

Esto indicaría un aumento del 3.5% en el acceso a Internet en esa provincia para el próximo trimestre.

---

In [11]:
# Mostrar la mediana y el DataFrame de resultados
print(f"Mediana de accesos por cada 100 hogares (2024, Trimestre 1): {mediana_velocidad}")
df_kpi_resultado_velocidad.head()


Mediana de accesos por cada 100 hogares (2024, Trimestre 1): 87.89500000000001


,Provincia,Accesos por cada 100 hogares,Nuevo_acceso,Mbps (Media de bajada),Nuevo_Mbps,KPI_Aumento_Acceso,KPI_Aumento_velocidad
0,Buenos Aires,81.10,82.7220,159.13,164.69955,2.0,3.5
1,Capital Federal,119.53,121.9206,229.94,237.98790,2.0,3.5
2,Catamarca,68.82,70.1964,91.45,94.65075,2.0,3.5
3,Chaco,46.70,47.6340,107.51,111.27285,2.0,3.5
4,Chubut,84.89,86.5878,20.50,21.21750,2.0,3.5


In [ ]:
df_kpi_resultado_velocidad.to_parquet('../Archivos_Finales/acceso_por_prov_poblacion_hogares', index=False)
df_kpi_resultado_fibra.to_parquet('../Archivos_Finales/accesos_totales_por_prov_por_tec', index=False)